# Die ROC-Curve - Beurteilung binärer Classifier
### Einführungsbeispiel

In [ ]:
import numpy as np
from sklearn import metrics
#Die tatsächliche Klassenzugehörigkeit (also 0 oder 1) ist hier wiedergegeben:
y = np.array([0,0,0,0,0,1,1,1,1,1]) 
# Gegeben seien Vorhersagen eines Modells: 0.5 sei die Entscheidungsgrenze: 
scores = np.array([0.45, 0.43, 0.55, 0.45, 0.55, 0.55, 0.6, 0.52, 0.51, 0.6])
#y_test_pred = (scores >= 0.9)
print('Das Modell sagt: ' + str(y_test_pred))
print('Tatsächlich:     ' + str(y == 1))
# ...um 0.5 herum gibt es also FP's und FN's, was mit der ROC-Curve gezeigt wird:
fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=1)
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.show()

In [ ]:
import pandas as pd
# Schöner in Seaborn.....
import seaborn as sn
import matplotlib.pyplot as plt
# y: tatsächlich              
conf_mat = pd.crosstab(y, y_test_pred, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(conf_mat, annot=True)
plt.show()

### Die ROC-Curve an einem Beispiel-Modell

In [ ]:
import pandas as pd

df = pd.read_csv("../classification.csv")

# Wenn du ein paar Spalten vorab aus den Daten entfernen
# df = df.drop("Spaltenname", axis = 1)

# Wenn du eine kategorische Variable in mehrere Spalten umwandeln
# möchtest, kannst du das mit folgendem Code tun:
# df = pd.get_dummies(df, columns = ["Spaltenname"])

df.size

In [ ]:
from sklearn.model_selection import train_test_split

# Welche Spalten sollen zur Vorhersage verwendet werden
X = df[["age", "interest"]].values

# Oder: Die Spalte "success" soll nicht zur Vorhersage verwendet werden:
# X = df.drop("success", axis = 1).values

y = df["success"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

In [ ]:
### Achtung!!! - Entweder Naiver-Bayes oder....
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

In [ ]:
### Achtung!!! - ....oder Logistic Regression ausführen...
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)
# das liefert aber nur 0en und 1en
y_pred

In [ ]:
#predict_proba() liefert nun Wahrscheinlichkeitswerte
#[....[Wahrscheinlichkeit0,  Wahrscheinlichkeit1], ....]
model.predict_proba(X_test)

In [ ]:

y_test_pred = model.predict_proba(X_test)[:, 1]

In [ ]:
y_test_pred

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred, pos_label=1)


In [ ]:
fpr

In [ ]:
tpr

In [ ]:
thresholds

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.show()

In [ ]:
roc_auc_score(y_test, y_test_pred)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.show()

In [ ]:
roc_auc_score(y_test, y_test_pred)

In [ ]:
metrics.roc_curve?

### Die Veranschaulichung der ROC-Curve mit mehr oder weniger aussagekräftigen Verteilungen von Score-Werten


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from sklearn import metrics
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
%matplotlib inline

n = 300
negative = np.random.normal(0, 0.1, n) #Gauss-Verteilung mit Mittelwert, Standardabweichung, Anzahl n Datenpunkte
positive = np.random.normal(0, 0.1, n) #Gauss-Verteilung mit Mittelwert, Standardabweichung, Anzahl n Datenpunkte

def hist_model(y_negativ, y_positiv):
    fig, axs = plt.subplots(1,3, figsize=(15, 5))
    fig.tight_layout(pad=5.0) #Abstand der Subplots
    #fig.suptitle('Titel')
    
    axs[0].set_title('Daten binärer Klassenzugehörigkeit')
    axs[0].set_xlim([0, 1])
    neg = negative+y_negativ
    pos = positive+y_positiv
    X = np.concatenate((neg, pos))
    y = np.concatenate(( np.zeros(n), np.ones(n) ))
    entscheidungsgrenze = (y_positiv + y_negativ) / 2
    y_pred = X >= entscheidungsgrenze
    axs[0].hist(neg, bins = (int)(n/10), color='red',  alpha = 0.5)
    axs[0].hist(pos, bins = (int)(n/10), color='blue', alpha = 0.5)
    axs[0].plot([entscheidungsgrenze,entscheidungsgrenze], [0,10], color='black')
    axs[0].text(entscheidungsgrenze, 0.44, 'Entscheidungsgrenze', color='black', horizontalalignment='center', verticalalignment='center', transform=axs[0].transAxes)
    axs[0].text(entscheidungsgrenze, 0.39, 'des Modells', color='black', horizontalalignment='center', verticalalignment='center', transform=axs[0].transAxes)

    axs[0].set_xlabel("Score-Wert des Modells", 
           #family='serif', 
           #color='r', 
           weight='normal', 
           size = 16,
           labelpad = 6)
    axs[0].set_ylabel("Anzahl", 
           #family='serif', 
           #color='r', 
           weight='normal', 
           size = 16,
           labelpad = 6)
    axs[0].text(0.5, 0.9, 'Rot: negativ', color='red', horizontalalignment='center', verticalalignment='center', transform=axs[0].transAxes)
    axs[0].text(0.5, 0.85, 'Blau: positiv', color='blue', horizontalalignment='center', verticalalignment='center', transform=axs[0].transAxes)
    axs[0].text(0.5, 0.8, 'Violett: uneindeutig', color='purple', horizontalalignment='center', verticalalignment='center', transform=axs[0].transAxes)

    axs[1].set_title('ROC-Kurve')   
    axs[1].set_xlabel("P(TP)", 
           #family='serif', 
           #color='r', 
           weight='normal', 
           size = 16,
           labelpad = 6)
    axs[1].set_ylabel("P(FP)", 
           #family='serif', 
           #color='r', 
           weight='normal', 
           size = 16,
           labelpad = 6)
    fpr, tpr, thresholds = metrics.roc_curve(y, X, pos_label=1)
    axs[1].plot(fpr, tpr)
    auc = roc_auc_score(y, y_pred)
    axs[1].text(0.5, 0.5, 'AUC-Score: {auc:.3f}'.format(auc=auc), horizontalalignment='center', verticalalignment='center', transform=axs[1].transAxes)

    axs[2].set_title('Confusion-Matrix')
    conf_mat = pd.crosstab(y, y_pred, rownames=['Actual'], colnames=['Predicted'])
    sn.heatmap(conf_mat, annot=True, ax=axs[2]) 
    
interact(hist_model, y_negativ = widgets.FloatSlider(value=0.35,
                                               min=0,
                                               max=1.0,
                                               step=0.05),
                    y_positiv = widgets.FloatSlider(value=0.65,
                                               min=0,
                                               max=1.0,
                                               step=0.05))